In [ ]:
import pandas as pd
import datetime as dt
from fmiopendata.wfs import download_stored_query
from tqdm import tqdm

#### without retry policy, 6 days intervals

In [72]:
import datetime as dt
from fmiopendata.wfs import download_stored_query

start_date = dt.date(2023, 10, 1)  # Example start date
end_date = dt.date(2023, 12, 31)

data_list = []  # List to collect data

current_date = start_date
while current_date <= end_date:
    # Set the start time to the current date
    start_time = current_date.isoformat() + "T00:00:00Z"
    # Calculate the end time 7 days from the start time
    end_date_interval = current_date + dt.timedelta(days=6)
    # Make sure the end date does not exceed the overall end date
    if end_date_interval > end_date:
        end_date_interval = end_date
    end_time = end_date_interval.isoformat() + "T23:59:59Z"

    print(f"Downloading data from {start_time} to {end_time}")  # Optional: print statement to track progress

    # Download data for the current 7-day interval
    obs = download_stored_query("fmi::observations::weather::multipointcoverage",
                                args=["bbox=18,55,35,75",  # whole Finland
                                      "starttime=" + start_time,
                                      "timestep="+str(60*24),  # daily entries
                                      "endtime=" + end_time,
                                      "timeseries=True"])

    # Parse and organize the data
    for station, station_data in obs.data.items():
        times = station_data['times']
        for param, values in station_data.items():
            if param != 'times':  # Skip the 'times' key
                for time, value in zip(times, values['values']):
                    data_list.append({'Timestamp': time, 'Station': station, param: value})

    # Move to the next 7-day interval for the next iteration
    current_date += dt.timedelta(days=6)


In [ ]:
import datetime as dt
from fmiopendata.wfs import download_stored_query
import time  # For adding delay between retries

start_date = dt.date(2023, 10, 1)  # Example start date
end_date = dt.date(2023, 12, 31)

data_list = []  # List to collect data

current_date = start_date
while current_date <= end_date:
    # Set the start time to the current date
    start_time = current_date.isoformat() + "T00:00:00Z"
    # Calculate the end time 7 days from the start time
    end_date_interval = current_date + dt.timedelta(days=6)
    # Make sure the end date does not exceed the overall end date
    if end_date_interval > end_date:
        end_date_interval = end_date
    end_time = end_date_interval.isoformat() + "T23:59:59Z"

    print(f"Downloading data from {start_time} to {end_time}")  # Optional: print statement to track progress

    max_retries = 5  # Maximum number of retries for each download attempt
    retry_delay = 5  # Delay between retries in seconds

    for attempt in range(max_retries):
        try:
            # Download data for the current 7-day interval
            obs = download_stored_query("fmi::observations::weather::multipointcoverage",
                                        args=["bbox=18,55,35,75",  # whole Finland
                                              "starttime=" + start_time,
                                              "timestep=" + str(60*24),  # daily entries
                                              "endtime=" + end_time,
                                              "timeseries=True"])
            # If download is successful, break out of the retry loop
            break
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)  # Wait before retrying
            else:
                print("Max retries reached, moving to next interval")
                obs = None  # Set obs to None to handle the case where all retries fail

    if obs is not None:
        # Parse and organize the data only if the download was successful
        for station, station_data in obs.data.items():
            times = station_data['times']
            for param, values in station_data.items():
                if param != 'times':  # Skip the 'times' key
                    for time, value in zip(times, values['values']):
                        data_list.append({'Timestamp': time, 'Station': station, param: value})

    # Move to the next 7-day interval for the next iteration
    current_date += dt.timedelta(days=7)  # Fixed to add 7 instead of 6 to avoid overlapping dates


#### data sorted by measurement type

In [ ]:
# Identify all unique data types (excluding 'Timestamp' and 'Station')
data_types = set(key for entry in data_list for key in entry if key not in ['Timestamp', 'Station'])

# Initialize a dictionary to hold a DataFrame for each data type
dfs = {}

# Extract unique station names and timestamps
station_names = sorted(set(entry['Station'] for entry in data_list))
timestamps = sorted(set(entry['Timestamp'] for entry in data_list))

# Create a DataFrame for each data type
for data_type in data_types:
    # Filter entries for the current data type
    filtered_data = [
        {key: value for key, value in entry.items() if key in ['Timestamp', 'Station', data_type]}
        for entry in data_list if data_type in entry
    ]

    # Initialize an empty DataFrame for the current data type
    df = pd.DataFrame(columns=station_names, index=pd.to_datetime(timestamps))

    # Fill the DataFrame with the current data type's measurements
    for entry in filtered_data:
        timestamp = entry['Timestamp']
        station = entry['Station']
        value = entry.get(data_type)  # Use .get() to handle missing data_type in some entries
        df.at[timestamp, station] = value

    # Filter rows that have data from at least half of the measurement stations
    threshold = len(station_names) // 2  # At least half of the stations must have data
    df_filtered = df.dropna(thresh=threshold)

    # Store the filtered DataFrame in the dictionary
    dfs[data_type] = df_filtered

# Access a specific filtered DataFrame by its data type, for example:
dfs['Wind speed']  # For filtered wind speed data

In [ ]:
import os

# Define the subfolder name
subfolder = 'data_by_measurement_type'

# Check if the subfolder exists, and if not, create it
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

# Iterate over the dfs dictionary to save each DataFrame to a CSV file in the subfolder
for data_type, df in dfs.items():
    # Format the data_type string to create a valid and readable filename
    filename = f"{data_type.replace(' ', '_').lower()}_data.csv"
    # Create the full path by joining the subfolder and filename
    full_path = os.path.join(subfolder, filename)
    
    # Save the DataFrame to a CSV file at the full path
    df.to_csv(full_path)
    
    print(f"Saved {data_type} data to {full_path}")


#### Data grouped by the measurement station

In [ ]:

# Identify all unique stations and measurement types
stations = sorted(set(entry['Station'] for entry in data_list))
measurement_types = sorted(set(key for entry in data_list for key in entry if key not in ['Timestamp', 'Station']))

# Initialize a dictionary to hold a DataFrame for each station
station_dfs = {}

# Create a DataFrame for each station
for station in stations:
    # Filter entries for the current station
    station_data = [entry for entry in data_list if entry['Station'] == station]
    
    # Extract timestamps for the current station
    timestamps = sorted(set(entry['Timestamp'] for entry in station_data))
    
    # Initialize an empty DataFrame for the current station
    df = pd.DataFrame(index=pd.to_datetime(timestamps), columns=measurement_types)
    
    # Fill the DataFrame with measurements
    for entry in station_data:
        timestamp = entry['Timestamp']
        for measurement in measurement_types:
            if measurement in entry:
                df.at[timestamp, measurement] = entry[measurement]

    # Store the DataFrame in the dictionary
    station_dfs[station] = df

# Define the subfolder name
subfolder = 'data_by_station'

# Check if the subfolder exists, and if not, create it
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

# Save each station's DataFrame to a CSV file in the subfolder
for station, df in station_dfs.items():
    # Format the station name to create a valid and readable filename
    filename = f"{station.replace(' ', '_').replace('/', '_').lower()}.csv"
    full_path = os.path.join(subfolder, filename)
    
    # Save the DataFrame to a CSV file
    df.to_csv(full_path)
    
    print(f"Saved data for {station} to {full_path}")